# Bezmerizing demo notebook

By [Allison Parrish](https://www.decontextualize.com/)

This notebook demonstrates some of the features of Bezmerizing.

See also [Flat and asemic writing](https://github.com/aparrish/material-of-language/blob/master/flat-asemic-writing.ipynb) for an introduction to the `Polyline` object.

In [1]:
%load_ext autoreload
%autoreload 2

In [27]:
import bezmerizing
from bezmerizing import Polyline, Bezier, QuadraticBezier

In [3]:
from flat import document, rgba, shape

In [4]:
import numpy as np

In [5]:
from IPython.display import SVG, Image, display

In [6]:
from IPython.display import SVG, display
def show(page):
    display(SVG(page.svg()))

## Bezier objects

A `Bezier` object represents a cubic Bézier curve.

In [9]:
bez = Bezier(
    [25, 25], # start
    [150, 50], # control point 1
    [50, 150], # control point 2
    [175, 175] # end
)

In [10]:
d = document(200, 200, 'pt')
page = d.addpage()
curve_fig = shape().stroke(rgba(0, 0, 0, 255)).nofill().width(4)
point_fig = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(255, 255, 255, 255)).width(2)
page.place(curve_fig.path(bez.to_path()))
page.place(point_fig.circle(*bez.start, 5))
page.place(point_fig.circle(*bez.cp1, 5))
page.place(point_fig.circle(*bez.cp2, 5))
page.place(point_fig.circle(*bez.end, 5))
show(page)

The `.point()` method returns the `[x, y]` coordinates of a point at the specified time on the curve (from 0 to 1):

In [11]:
n_pts = 12
pts = []
for i in range(n_pts + 1):
    pts.append(bez.point(i/n_pts))

In [12]:
pts

[[25.0, 25.0],
 [51.822916666666664, 32.725694444444436],
 [70.83333333333334, 43.05555555555556],
 [83.59375, 55.46875],
 [91.66666666666667, 69.44444444444444],
 [96.6145833333333, 84.46180555555554],
 [100.0, 100.0],
 [103.38541666666667, 115.53819444444447],
 [108.33333333333331, 130.55555555555554],
 [116.40625, 144.53125],
 [129.16666666666669, 156.94444444444446],
 [148.17708333333331, 167.27430555555554],
 [175.0, 175.0]]

In [13]:
d = document(200, 200, 'pt')
page = d.addpage()
curve_fig = shape().stroke(rgba(0, 0, 0, 255)).nofill().width(4)
point_fig = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(255, 255, 255, 255)).width(2)
page.place(curve_fig.path(bez.to_path()))
for pt in pts:
    page.place(point_fig.circle(*pt, 5))
show(page)

### Curve offsets

The `.offsets()` method returns a polyline with points at the given distance from the curve:

In [28]:
offset_p = bez.offsets(np.linspace(4, 16, 12))

In [29]:
offset_p

Polyline([[24.2155, 28.9223], [51.8557, 38.2488], [69.6672, 49.9649], [80.1047, 63.1229], [85.8499, 77.5715], [89.0117, 93.4870], [91.4390, 110.7346], [95.2698, 129.0268], [103.1481, 147.8101], [117.6981, 165.5388], [140.3077, 180.1519], [171.8621, 190.6893]])

In [30]:
d = document(200, 200, 'pt')
page = d.addpage()
curve_fig = shape().stroke(rgba(0, 0, 0, 255)).nofill().width(4)
point_fig = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(255, 255, 255, 255)).width(2)
offset_fig = shape().stroke(rgba(0, 0, 255, 255)).nofill().width(2)
page.place(curve_fig.path(bez.to_path()))
page.place(offset_fig.polyline(offset_p))
for pt in offset_p.vertices:
    page.place(point_fig.circle(*pt, 2))
show(page)

The `.offset_polygon()` method returns a polyline with the given offsets on both sides of the curve:

In [31]:
poly = bez.offset_polygon(np.linspace(8, 16, 20))

In [32]:
d = document(200, 200, 'pt')
page = d.addpage()
curve_fig = shape().stroke(rgba(0, 0, 0, 255)).nofill().width(4)
point_fig = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(255, 255, 255, 255)).width(2)
offset_fig = shape().stroke(rgba(0, 0, 255, 255)).fill(rgba(0, 0, 0, 40)).width(2)
page.place(curve_fig.path(bez.to_path()))
page.place(offset_fig.polygon(poly))
for pt in poly.vertices:
    page.place(point_fig.circle(*pt, 2))
show(page)

The `QuadraticBezier` class has the same interface as `Bezier`, and operates on quadratic Béziers (only one control point) instead of cubic Béziers:

In [35]:
qbez = QuadraticBezier(
    [25, 25], # start
    [150, 50], # control point 1
    [50, 150] # end
)
qpoly = qbez.offset_polygon(np.linspace(8, 16, 20))

In [37]:
d = document(200, 200, 'pt')
page = d.addpage()
curve_fig = shape().stroke(rgba(0, 0, 0, 255)).nofill().width(4)
point_fig = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(255, 255, 255, 255)).width(2)
offset_fig = shape().stroke(rgba(0, 0, 255, 255)).fill(rgba(0, 0, 0, 40)).width(2)
page.place(curve_fig.path(qbez.to_path()))
page.place(offset_fig.polygon(qpoly))
for pt in qpoly.vertices:
    page.place(point_fig.circle(*pt, 2))
show(page)

## Polylines

A `Polyline` object stores a sequence of points. The object has a special iteration handler, which makes it possible to pass the object directly to Flat's `.polyline()` method:

In [44]:
random_p = Polyline([[np.random.normal(100, 50), np.random.normal(100, 50)] for i in range(12)])

In [45]:
d = document(200, 200, 'pt')
page = d.addpage()
line_fig = shape().stroke(rgba(0, 0, 0, 255)).nofill().width(1)
point_fig = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(255, 255, 255, 255)).width(1)
page.place(line_fig.polyline(random_p))
for pt in random_p.vertices:
    page.place(point_fig.circle(*pt, 2))
show(page)

## Catmull-Rom splines from polylines

The polyline's `.catmull_spline()` method returns a `Spline` object composed of `Bezier` objects that result from drawing a Catmull-Rom spline through the polyline:

In [50]:
spline = random_p.catmull_spline(tightness=0.1)

In [51]:
len(spline.beziers)

9

In [52]:
path = spline.to_path()

In [53]:
d = document(200, 200, 'pt')
page = d.addpage()
curve_fig = shape().stroke(rgba(0, 0, 255, 255)).nofill().width(2)
page.place(curve_fig.path(path))
show(page)

The `.smooth_path()` function returns a `Path` directly.

In [54]:
path = random_p.augment().smooth_path(tightness=0.1)

In [55]:
d = document(200, 200, 'pt')
page = d.addpage()
curve_fig = shape().stroke(rgba(0, 0, 255, 255)).nofill().width(2)
page.place(curve_fig.path(path))
show(page)

In [56]:
d = document(200, 200, 'pt')
page = d.addpage()
line_fig = shape().stroke(rgba(0, 0, 0, 255)).nofill().width(1)
curve_fig = shape().stroke(rgba(0, 0, 255, 255)).nofill().width(2)
point_fig = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(255, 255, 255, 255)).width(1)
page.place(line_fig.polyline(random_p))
page.place(curve_fig.path(path))
for pt in random_p.vertices:
    page.place(point_fig.circle(*pt, 2))
show(page)

## Offsets of splines

The `Spline` class has a `.tangent_offsets()` method that returns a polyline from the concatenated result of calling `.offsets()` on each curve in the spline. The `distances` parameter sets the offset distance along the spline, and the `samples_per` parameter specifies the number of points to sample along each curve:

In [57]:
polyline = spline.tangent_offsets(
    distances=np.sin(np.linspace(0, np.pi*50, 500))*6,
    samples_per=48
)
curve_path = spline.to_path()

In [58]:
d = document(200, 200, 'pt')
page = d.addpage()
polyline_fig = shape().stroke(rgba(255, 40, 40, 255)).width(2)
curve_fig = shape().stroke(rgba(128, 128, 255, 255))
page.place(polyline_fig.polyline(polyline))
page.place(curve_fig.path(curve_path))
show(page)

The `.tangent_offset_polygon()` works the same way, but also includes a polyline drawn with the same parameters on the other side of the curve:

In [59]:
poly = spline.tangent_offset_polygon(
    thicknesses=[2, 8, 2],
    samples_per=12)

In [60]:
d = document(200, 200, 'pt')
page = d.addpage()
curve_fig = shape().fill(rgba(40, 40, 40, 255)).nostroke()
page.place(curve_fig.polygon(poly))
show(page)

## Putting everything together

The `.fancy_curve()` method of a `Polyline` object returns a `Polyline` that results from converting the `Polyline` object to a spline (with Catmull-Rom), then calling `.tangent_offset_polygon()` on the resulting spline.

In [63]:
pts = [[np.random.normal(300, 100), np.random.normal(300, 100)] for i in range(24)]

In [64]:
thicknesses = (np.sin(np.linspace(0, np.pi*4, 100))+1) * 8

In [65]:
poly = Polyline(pts).fancy_curve(thicknesses=thicknesses, samples_per=24)

In [66]:
d = document(600, 600, 'pt')
page = d.addpage()
curve_fig = shape().fill(rgba(40, 40, 40, 255)).nostroke()
page.place(curve_fig.polygon(poly))
show(page)

## Path class

A `Path` can combine lines, curves and movement (using objects from Flat's `command` module). `Path` objects can be rotated, scaled, and translated just like their component curves and lines. You can pass a `Path` object directly to Flat's `.path()` method:

In [78]:
from bezmerizing import Path
from flat import command

In [79]:
p = Path([
    command.moveto(10, 10),
    command.lineto(15, 25),
    command.curveto(15, 30, 5, 30, 5, 25),
    command.closepath
])

In [80]:
d = document(100, 100, 'mm')
page = d.addpage()
fig = shape().stroke(rgba(40, 40, 40, 255)).nofill()
page.place(fig.path(p))
page.place(fig.path(p.translate(20, 20)))
page.place(fig.path(p.translate(20, 20).rotate(np.pi*0.125)))
page.place(fig.path(p.translate(20, 20).rotate(np.pi*0.125).scale(1.5)))
show(page)

The string representation of a `Path` shows the coordinates of each component command:

In [81]:
p

Path([moveto(x=10.0000, y=10.0000), lineto(x=15.0000, y=25.0000), curveto(x1=15.0000, y1=30.0000, x2=5.0000, y2=30.0000, x=5.0000, y=25.0000), closepath])

## Polyline primitives

Bezmerizing has several functions for creating `Polyline` objects from geometrical primitives:

In [82]:
from bezmerizing.primitives import arc, ellipse, circle, rect

In [83]:
print(arc(0, 0, 100, 150, 0, (3/2)*np.pi).translate(50, 50))

Polyline([[150.0000, 50.0000], [149.7204, 61.2095], [148.8831, 72.3563], [147.4928, 83.3781], [145.5573, 94.2133], [143.0874, 104.8012], [140.0969, 115.0826], [136.6025, 125.0000], [132.6239, 134.4980], [128.1831, 143.5235], [123.3052, 152.0259], [118.0173, 159.9578], [112.3490, 167.2747], [106.3320, 173.9358], [100.0000, 179.9038], [93.3884, 185.1453], [86.5341, 189.6311], [79.4755, 193.3359], [72.2521, 196.2392], [64.9042, 198.3246], [57.4730, 199.5806], [50.0000, 200.0000], [42.5270, 199.5806], [35.0958, 198.3246], [27.7479, 196.2392], [20.5245, 193.3359], [13.4659, 189.6311], [6.6116, 185.1453], [-0.0000, 179.9038], [-6.3320, 173.9358], [-12.3490, 167.2747], [-18.0173, 159.9578], [-23.3052, 152.0259], [-28.1831, 143.5235], [-32.6239, 134.4980], [-36.6025, 125.0000], [-40.0969, 115.0826], [-43.0874, 104.8012], [-45.5573, 94.2133], [-47.4928, 83.3781], [-48.8831, 72.3563], [-49.7204, 61.2095], [-50.0000, 50.0000], [-49.7204, 38.7905], [-48.8831, 27.6437], [-47.4928, 16.6219], [-45.55

In [84]:
a = arc(0, 0, 40, 60, 0, (3/2)*np.pi)
b = ellipse(0, 0, 60, 40)
c = circle(0, 0, 75)
d = rect(-40, -25, 80, 50)

In [85]:
doc = document(400, 400, 'pt')
page = doc.addpage()
curve_fig = shape().fill(rgba(0, 128, 128, 128)).stroke(rgba(0, 0, 0, 255)).width(2)
page.place(curve_fig.polyline(a.translate(50, 200)))
page.place(curve_fig.polygon(b.translate(150, 200)))
page.place(curve_fig.polygon(c.translate(250, 200)))
page.place(curve_fig.polygon(d.translate(350, 200)))
show(page)

In [86]:
b = ellipse(200, 200, 140, 100)
doc = document(400, 400, 'pt')
page = doc.addpage()
curve_fig = shape().fill(rgba(0, 128, 128, 128)).stroke(rgba(0, 0, 0, 255)).width(2)
point_fig = shape().fill(rgba(255, 255, 255, 255)).stroke(rgba(0, 0, 0, 255)).width(1)
page.place(curve_fig.polyline(b))
for pt in b.vertices:
    page.place(point_fig.circle(*pt, 3))
show(page)

You can make polygons with sides of regular length using the `circle` or `ellipse` functions, and setting the `n` parameter to number of points in the desired polygon:

In [88]:
b = ellipse(200, 200, 140, 100, n=5)

In [89]:
doc = document(400, 400, 'pt')
page = doc.addpage()
curve_fig = shape().fill(rgba(0, 128, 128, 128)).stroke(rgba(0, 0, 0, 255)).width(2)
point_fig = shape().fill(rgba(255, 255, 255, 255)).stroke(rgba(0, 0, 0, 255)).width(1)
page.place(curve_fig.polyline(b))
for pt in b.vertices:
    page.place(point_fig.circle(*pt, 3))
show(page)

You can call the `.fancy_curve()` method on polylines constructed with this method:

In [90]:
doc = document(400, 400, 'pt')
page = doc.addpage()
fig = shape().fill(rgba(40, 40, 40, 255)).nostroke()
grid_size = 9
grid_res = doc.width / grid_size
for i in range(grid_size):
    for j in range(grid_size):
        for k in range(np.random.choice([2, 3, 4, 5])):
            poly = ellipse(0, 0,
                           np.random.uniform((doc.width/grid_size)*0.4),
                           np.random.uniform((doc.height/grid_size)*0.4),
                           n=np.random.choice([4, 5, 6, 7]))
            poly_fancy = poly.fancy_curve(thicknesses=[1, 2, 1, 2, 1],
                                          interp="cubic",
                                          tightness=0.5)
            page.place(fig.polygon(poly_fancy.rotate(np.random.uniform(np.pi*2)).translate(
                i*grid_res+grid_res*0.5, j*grid_res+grid_res*0.5)))
show(page)

In [91]:
parts = []
for i in range(5):
    arc_poly = arc(0, 0, 100, 100, np.random.uniform(0, np.pi*2), np.random.uniform(0, np.pi*2))
    poly = arc_poly.fancy_curve(thicknesses=[10, 5, 1, 0], interp="quadratic")
    poly = poly.scale(np.random.uniform(0.25, 1.5))
    parts.append(poly)
d = document(400, 400, 'pt')
page = d.addpage()
curve_fig = shape().fill(rgba(40, 40, 40, 255)).nostroke()
for part in parts:
    page.place(curve_fig.polygon(part.translate(200, 200)))
show(page)